## Building a Multi Search Agent 

In [ ]:
!pip install arxiv

]11;?\Collecting arxiv
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6089 sha256=26104ea7b7950f33a717ceb385ffcbd7ea8854b3e624119a1286d552150526cc
  Stored in directory: /home/aditya/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [43]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI



In [57]:
import os
from dotenv import load_dotenv
load_dotenv()
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
llm = ChatOpenAI(model="gpt-4",api_key=OPEN_AI_API_KEY, temperature=0.5)

In [47]:
api= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki= WikipediaQueryRun(api_wrapper=api)

In [48]:
wiki.name  #This is the first tool in the community

'wikipedia'

## Adding Web Base Loader to retrieve information from website

In [49]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#url= input("Enter the URL of the document you want to summarize: ")
loader = WebBaseLoader("https://python.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents(docs)
db = FAISS.from_documents(documents, OpenAIEmbeddings(api_key=OPEN_AI_API_KEY))   
retriever= db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x70f5a43ba660>, search_kwargs={})

In [50]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langchain_search","Search for the information about the langchain. For any question about Langchain, you must use this tool.")

In [51]:
retriever_tool.name  #This is the second tool in the community

'langchain_search'

In [52]:
## Creating an Arxiv API Wrapper.
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api= ArxivAPIWrapper(top_k_results=2, load_content_chars_max=200) #This will load the top 2 results from the arxiv API
arxiv= ArxivQueryRun(api_wrapper=arxiv_api)
arxiv.name    


'arxiv'

In [53]:
tools=[wiki,arxiv,retriever_tool]   
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/aditya/Langflow(RAG)/lang/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langchain_search', description='Search for the information about the langchain. For any question about Langchain, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x70f5ca167600>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_c

In [ ]:
from langchain import hub

In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools),prompt=  

TypeError: create_openai_tools_agent() missing 1 required positional argument: 'prompt'